## Benders Decomp 

$\theta$ represents to cost of assigning location i to facility j

## Objective Function
$$
\min \sum_{j \in N} f_j y_j + \sum_{i \in N} \theta_i
$$


### Inital Problem (Open atleast 1 facility)
$$
s.t. \sum_{j \in N} \geq 1
$$

$$
\theta_i \geq \min_{j \in N} \{c_{ij}\} \forall i \in N
$$

### Solve (Assign it to the closest Open facility)
$$
\Theta_i (y^*) = \min_{j \in N | y^*_i = 1} c_{ij}
$$

$$
\theta_i \geq \Theta_i(y^*) - \sum_{j \in N | c_{ij} < \Theta_i (y^*)} (\Theta_i (y^*) - c_{ij})
$$

In [5]:
import math
import random
import gurobipy as gp

EPS = 1e-6

random.seed(3)

nLocs = 300
I = range(nLocs)
J = range(nLocs)
F = [random.randint(10000,20000) for i in I]
C = [[random.randint(1000,2000) for j in J] for i in I]

m = gp.Model()

# Vars
Y = {j: m.addVar(vtype=gp.GRB.BINARY) for j in J}
Theta = {i: m.addVar(lb=min(C[i])) for i in I}

OpenAtLeastOne = m.addConstr(gp.quicksum(Y.values()) >= 1)

m.setObjective(
    gp.quicksum(F[j] * Y[j] for j in J)
    + gp.quicksum(Theta.values())
)

for k in range(10):
    m.optimize() 

    # Set of open facilities
    FSet = {j for j in J if round(Y[j].x)==1}

    # Cost of opening things and the cost of assigning things
    UpperBound = sum(F[j] for j in FSet)
    
    CutsAdded = 0
    for i in I:
        cMin = min(C[i][j] for j in FSet)
        UpperBound += cMin
        if cMin > Theta[i].x + EPS:
            CutsAdded += 1
            # Increase theta to atleast this cost unless we open a facility that is closer
            m.addConstr(
                Theta[i] >= cMin - gp.quicksum(
                    (cMin - C[i][j]) * Y[j] 
                    for j in J
                    if C[i][j] < cMin
                )
            )
    print('*' * 50)
    print(k, CutsAdded, round(m.ObjVal),  UpperBound, FSet)
    print('*' * 50)



Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 600 columns and 300 nonzeros
Model fingerprint: 0x460e0c0d
Variable types: 300 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 312790.00000
Presolve removed 1 rows and 600 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 310778 312790 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.107780000000e+05, best bound 3.107780000000e+05, gap 0.0000%
**************************************************
0 299 310778 456436 {295}

In [16]:
import math
import random
import gurobipy as gp

EPS = 1e-6

random.seed(3)

nLocs = 300
I = range(nLocs)
J = range(nLocs)
F = [random.randint(10000,20000) for i in I]
C = [[random.randint(1000,2000) for j in J] for i in I]

m = gp.Model()

# Vars
Y = {j: m.addVar(vtype=gp.GRB.BINARY) for j in J}
Theta = {i: m.addVar(lb=min(C[i])) for i in I}

OpenAtLeastOne = m.addConstr(gp.quicksum(Y.values()) >= 1)

m.setObjective(
    gp.quicksum(F[j] * Y[j] for j in J)
    + gp.quicksum(Theta.values())
)

def Callback(model, where):
    if where == gp.GRB.Callback.MIPSOL:
        YV = model.cbGetSolution(Y)
        ThetaV = model.cbGetSolution(Theta)


        # Set of open facilities
        FSet = {j for j in J if round(YV[j])==1}

        # Cost of opening things and the cost of assigning things
        UpperBound = sum(F[j] for j in FSet)
        
        CutsAdded = 0
        for i in I:
            cMin = min(C[i][j] for j in FSet)
            UpperBound += cMin
            if cMin > ThetaV[i] + EPS:
                ThetaV[i] = cMin
                CutsAdded += 1
                # Increase theta to atleast this cost unless we open a facility that is closer
                model.cbLazy(
                    Theta[i] >= cMin - gp.quicksum(
                        (cMin - C[i][j]) * Y[j] 
                        for j in J
                        if C[i][j] < cMin
                    )
                )
        if CutsAdded > 0 and UpperBound < round(model.cbGet(gp.GRB.Callback.MIPSOL_OBJBST)):
            print(f'Found {UpperBound}')
            model.cbSetSolution(Y, YV)
            model.cbSetSolution(Theta, ThetaV)



# Make model continious
for j in J:
    Y[j].VType= gp.GRB.CONTINUOUS
    Y[j].ub = 1

# Warm Starting - get a possible list of facilities
ExtraCon = []
for k in range(10):
    m.optimize() 

    # Set of open facilities
    FSet = {j: Y[j].x for j in J if Y[j].x >= EPS}

    # Cost of opening things and the cost of assigning things
    UpperBound = sum(F[j] * FSet[j] for j in FSet)

    CutsAdded = 0
    for i in I:
        # Get sorted list of cheapest assignments
        cSort = sorted([(C[i][j], j) for j in FSet])

        AssignCost = 0.0
        LeftToAssign = 1.0
        for (c, j) in cSort:
            Use = min(FSet[j], LeftToAssign)
            AssignCost += Use * c
            LeftToAssign -= Use
            if LeftToAssign < EPS:
                cCrit = c
                break

        UpperBound += AssignCost
        if AssignCost > Theta[i].x + EPS:
            CutsAdded += 1
            # Increase theta to atleast this cost unless we open a facility that is closer
            ExtraCon.append(m.addConstr(
                Theta[i] >= cCrit - gp.quicksum(
                    (cCrit - C[i][j]) * Y[j] 
                    for j in J
                    if C[i][j] < cCrit
                )
            ))
    print('*' * 50)
    print(k, CutsAdded, round(m.ObjVal),  UpperBound)
    print('*' * 50)


# Remove any extra constraints with slack
for c in ExtraCon:
    if abs(c.Slack) > EPS:
        m.remove(c)

# Convert back
for j in J:
    Y[j].VType = gp.GRB.BINARY




m.Params.LazyConstraints = 1 # Lazy constrain will cut off Int solutions that were previously fesiable
m.Params.BranchDir = 1 # Can possibliy speed up run time
# m.Params.Seed = 1
m.optimize(Callback)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 600 columns and 300 nonzeros
Model fingerprint: 0x0687970a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 307 columns
Presolve time: 0.00s
Presolved: 1 rows, 293 columns, 293 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0074900e+05   1.000000e+00   0.000000e+00      0s
       1    3.1077800e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.107780000e+05
**************************************************
0 299 310778 456436.0
**************************************************
Gurobi Optimizer version 12.0.3 build v12.0.3